In [57]:
import csv
import numpy as np
import collections

In [83]:
data = []
counter = 0
with open('plays_clean.csv', 'r') as f:
    reader = csv.DictReader(f)
    for i,row in enumerate(reader):
        if not row['off'] and not row['def']:
            print(i, row)
            counter += 1
        data.append(row)
print(counter)

0


In [59]:
grouped_by_game = collections.defaultdict(list)
for line in data:
    grouped_by_game[line['gameid']].append(line)
grouped_by_game = list(grouped_by_game.values())

In [60]:
for game in grouped_by_game:
    for play_dict in game:
        for key in play_dict:
            try:
                play_dict[key] = int(play_dict[key])
            except:
                pass

In [61]:
def put_into_bucket(play_dict):
    yard = play_dict['ydline'] // 10
    down = play_dict['down']
    if play_dict['togo'] <= 2:
        togo = 0
    elif play_dict['togo'] <= 4:
        togo = 1
    elif play_dict['togo'] <= 6:
        togo = 2
    elif play_dict['togo'] <= 8:
        togo = 3
    elif play_dict['togo'] <= 10:
        togo = 4
    elif play_dict['togo'] <= 15:
        togo = 5
    elif play_dict['togo'] <= 20:
        togo = 6
    else:
        togo = 7
    if play_dict['off'] < play_dict['def']:
        posession = 1
    else:
        posession = 0
    return (yard, down, togo, posession)

In [62]:
grouped_by_game[0][0]

{'def': 'Jets',
 'defscore': 0,
 'description': '(14:55) V.Young pass short right to A.Hall pushed ob at TEN 48 for 11 yards (K.Rhodes).',
 'down': 1,
 'gameid': '20071223_NYJ@TEN',
 'off': 'Titans',
 'offscore': 0,
 'time': 3595,
 'togo': 10,
 'ydline': 63}

In [63]:
game = grouped_by_game[0]
scores = collections.defaultdict(list)
for play in game:
    scores[play['off']].append(play['offscore'])
    scores[play['def']].append(play['defscore'])
    
for i in range(1, len(game)):
    for team in scores:
        if scores[team][i] != scores[team][i -1]:
            print(i, game[i])

37 {'offscore': 7, 'defscore': 0, 'def': 'Jets', 'time': 2666, 'ydline': 4, 'gameid': '20071223_NYJ@TEN', 'down': '', 'togo': '', 'off': 'Titans', 'description': 'R.Bironas kicks 67 yards from TEN 30 to NYJ 3. L.Washington to NYJ 23 for 20 yards (B.Troupe M.Griffin).'}
46 {'offscore': 6, 'defscore': 7, 'def': 'Titans', 'time': 2465, 'ydline': 9, 'gameid': '20071223_NYJ@TEN', 'down': '', 'togo': '', 'off': 'Jets', 'description': 'M.Nugent kicks 70 yards from NYJ 30 to end zone Touchback.'}
95 {'offscore': 10, 'defscore': 6, 'def': 'Jets', 'time': 1300, 'ydline': 28, 'gameid': '20071223_NYJ@TEN', 'down': '', 'togo': '', 'off': 'Titans', 'description': 'R.Bironas kicks 58 yards from TEN 30 to NYJ 12. L.Washington to NYJ 32 for 20 yards (J.Cordova R.Scanlon).'}


In [64]:
counter = 0
features = ['down', 'togo']
for game in grouped_by_game:
    for i, play in enumerate(game):
        missing = False
        for feature in features:
            if not play[feature]:
                missing = True
        if missing:
            counter += 1
            scores = collections.defaultdict(set)
            num_gaps = 1
            for j in range(i-1, i + 1):
                if 0 <= j < len(game):
                    scores[game[j]['off']].add(game[j]['offscore'])
                    scores[game[j]['def']].add(game[j]['defscore'])
                    for feature in features:
                        if not game[j][feature]:
                            num_gaps += 1
            score_changed = False
            for team in scores:
                if len(scores) >= 1:
                    score_changed = True
                #if max(scores[team]) - min(scores[team]) in (2,3):
                #    num_gaps -= 1
                #if max(scores[team]) - min(scores[team]) in (6,7,8):
                #    num_gaps -= 2
            #if num_gaps > 0:
            #    #print(play)
            #    pass
            if not score_changed:
                print(play)

In [65]:
def group_by_score_change(game, original_scores = None):
    if not game:
        return []
    if original_scores is None:
        original_scores = collections.defaultdict(int)
    changed_team = None
    changed_score = None
    for i, play in enumerate(game):
        if original_scores[play['off']] != play['offscore']:
            changed_team = play['off']
            #changed_score = play['offscore'] 
            changed_score = play['offscore'] - original_scores[changed_team]
            break
        if original_scores[play['def']] != play['defscore']:
            changed_team = play['def']
            #changed_score = play['defscore'] 
            changed_score = play['defscore'] - original_scores[changed_team]
            break     
    
    if changed_team is None:
        return [('', 0,game)]
    
    new_scores = collections.defaultdict(int)
    new_scores[play['off']] = play['offscore']
    new_scores[play['def']] = play['defscore']
    return [(changed_team, changed_score, game[:i+1])] + group_by_score_change(game[i+1:], new_scores)
    
    
    '''if changed_score in (2,3):
        return [game[:i+1]] + group_by_score_change(game[i+1:, new_scores])
    
    if changed_score == 6:
        j = i + 1
        if j < len(game):'''
            
    

In [66]:
def bucketable(play):
    if not play['down']:
        return False
    if play['down'] < 1:
        return False
    if play['down'] > 4:
        return False
    if not play['togo']:
        return False
    if play['togo'] < 1:
        return False
    if not play['ydline']:
        return False
    if play['ydline'] < 1:
        return False
    if not play['off'] or not play['def']:
        return False
    return True
    


In [80]:
def possession(play):
    if play_dict['off'] < play_dict['def']:
        return 1
    else:
        return 0

def bucket_group(group):
    team, diff, group = group
    answer = []
    teams = set()
    for play in game:
        teams.add(play['off'])
        teams.add(play['def'])
    if '' in teams:
        teams.remove('')
    if team:
        pos = 1- sorted(teams).index(team)
    if diff != 0:
        group = group[:-1]
    for i, play in enumerate(group):
        if play['time'] in (1800, 3600):
            if i == 0 or group[i-1]['time'] in (1800, 3600):
                answer.append([6])
        if bucketable(play):
            answer.append(put_into_bucket(play))
        else:
            answer.append([6])
    if team:
        
        if answer[-1] == [6]:
            answer = answer[:-1]
        if diff == 2:
            answer.append([1, pos])
            answer.append([5, pos])
        elif diff == 3:
            answer.append([0, pos])
            answer.append([5, pos])
        elif diff == 6:
            answer.append([2, pos])
            answer.append([5, pos])
        elif diff == 7:
            answer.append([2, pos])
            answer.append([3, pos])
            answer.append([5, pos])
        elif diff == 8:
            answer.append([2, pos])
            answer.append([4, pos])
            answer.append([5, pos])
        elif diff != 0:
            answer.append([6])
    return answer
        

In [68]:
def bucket_game(game):
    grouped = group_by_score_change(game)
    answer = []
    for group in grouped:
        answer += bucket_group(group)
    return answer
        


In [69]:
group_by_score_change(grouped_by_game[0])[0]

('Titans',
 7,
 [{'def': 'Jets',
   'defscore': 0,
   'description': '(14:55) V.Young pass short right to A.Hall pushed ob at TEN 48 for 11 yards (K.Rhodes).',
   'down': 1,
   'gameid': '20071223_NYJ@TEN',
   'off': 'Titans',
   'offscore': 0,
   'time': 3595,
   'togo': 10,
   'ydline': 63},
  {'def': 'Jets',
   'defscore': 0,
   'description': '(14:31) L.White up the middle to NYJ 48 for 4 yards (K.Coleman).',
   'down': 1,
   'gameid': '20071223_NYJ@TEN',
   'off': 'Titans',
   'offscore': 0,
   'time': 3571,
   'togo': 10,
   'ydline': 52},
  {'def': 'Jets',
   'defscore': 0,
   'description': '(13:56) L.White left tackle to NYJ 37 for 11 yards (K.Rhodes).',
   'down': 2,
   'gameid': '20071223_NYJ@TEN',
   'off': 'Titans',
   'offscore': 0,
   'time': 3536,
   'togo': 6,
   'ydline': 48},
  {'def': 'Jets',
   'defscore': 0,
   'description': '(13:18) L.White left guard to NYJ 35 for 2 yards (S.Pouha D.Harris).',
   'down': 1,
   'gameid': '20071223_NYJ@TEN',
   'off': 'Titans',
 

In [70]:
bucket_game(grouped_by_game[1])

[(8, 1, 4, 0),
 (8, 2, 4, 0),
 (7, 3, 2, 0),
 (7, 4, 2, 0),
 (6, 1, 4, 1),
 (6, 2, 4, 1),
 (6, 3, 4, 1),
 (6, 4, 1, 1),
 (8, 1, 4, 0),
 (8, 2, 4, 0),
 (7, 1, 4, 0),
 (7, 2, 3, 0),
 (6, 3, 0, 0),
 (5, 1, 4, 0),
 (6, 2, 5, 0),
 (5, 3, 3, 0),
 (5, 4, 1, 0),
 (7, 1, 4, 1),
 (7, 2, 4, 1),
 (7, 3, 0, 1),
 (7, 4, 0, 1),
 (8, 1, 4, 0),
 (0, 1, 3, 1),
 (0, 2, 3, 1),
 [2, 1],
 [3, 1],
 [5, 1],
 (8, 1, 4, 0),
 (8, 2, 4, 0),
 (7, 1, 4, 0),
 (6, 2, 0, 0),
 (5, 1, 4, 0),
 (4, 1, 4, 0),
 (4, 2, 2, 0),
 (3, 1, 4, 0),
 (4, 1, 5, 0),
 (4, 2, 5, 0),
 (2, 3, 1, 0),
 (2, 4, 0, 0),
 (6, 1, 4, 1),
 (6, 2, 4, 1),
 (6, 3, 4, 1),
 (7, 3, 5, 1),
 (6, 4, 4, 1),
 (7, 1, 4, 0),
 (7, 2, 2, 0),
 (5, 1, 4, 0),
 (4, 1, 4, 0),
 (3, 2, 3, 0),
 (3, 3, 2, 0),
 (4, 3, 5, 0),
 (4, 4, 5, 0),
 (9, 1, 4, 1),
 (9, 1, 5, 1),
 [6],
 (7, 1, 4, 0),
 (6, 2, 0, 0),
 (6, 1, 4, 0),
 (6, 2, 4, 0),
 (5, 3, 2, 0),
 (7, 3, 6, 0),
 (6, 4, 4, 0),
 (8, 1, 4, 1),
 (8, 2, 5, 1),
 (8, 3, 5, 1),
 (6, 1, 4, 1),
 (7, 2, 6, 1),
 (7, 3, 5, 1),
 (7, 4,

In [71]:
for j, game in enumerate(grouped_by_game):
    scores = collections.defaultdict(list)
    for play in game:
        scores[play['off']].append(play['offscore'])
        scores[play['def']].append(play['defscore'])

    for i in range(2, len(game)):
        for team in scores:
            try:
                if len(set((scores[team][i], scores[team][i-1], scores[team][i-2]))) == 3:
                    print(j, i,scores[team][i], scores[team][i-1], scores[team][i-2])
            except:
                pass

341 111 11 10 3
950 9 9 7 0
2473 47 17 18 10
2814 2 17 10 3


In [72]:
x = []
for game in grouped_by_game:
    c = 0
    for play in game:
        if play['time'] == 1800:
            c += 1
    x.append(c)

In [81]:
bucket_sequence = []
for game in grouped_by_game:
    bucket_sequence += bucket_game(game)
    bucket_sequence += [6]

IndexError: list index out of range

In [76]:
group_by_score_change(game)[0]

('Bengals',
 3,
 [{'def': 'Chiefs',
   'defscore': 0,
   'description': '(14:56) J.Kitna pass incomplete to T.Stewart (S. Barber). Knocked down at the line of scrimmage.',
   'down': 1,
   'gameid': '20031116_KC@CIN',
   'off': 'Bengals',
   'offscore': 0,
   'time': 3596,
   'togo': 10,
   'ydline': 72},
  {'def': 'Chiefs',
   'defscore': 0,
   'description': '(14:50) P.Warrick left end pushed ob at CIN 39 for 11 yards (M. Maslowski). Reverse.',
   'down': 2,
   'gameid': '20031116_KC@CIN',
   'off': 'Bengals',
   'offscore': 0,
   'time': 3590,
   'togo': 10,
   'ydline': 72},
  {'def': 'Chiefs',
   'defscore': 0,
   'description': '(14:26) Ru.Johnson left tackle to CIN 41 for 2 yards (J. Browning).',
   'down': 1,
   'gameid': '20031116_KC@CIN',
   'off': 'Bengals',
   'offscore': 0,
   'time': 3566,
   'togo': 10,
   'ydline': 61},
  {'def': 'Chiefs',
   'defscore': 0,
   'description': '(13:52) Ru.Johnson right tackle to CIN 46 for 5 yards (J. Browning).',
   'down': 2,
   'gameid

In [82]:
grouped_by_game.index(game)

112

In [75]:
len(grouped_by_game)

2911

In [ ]:
for play